In [1]:
from astropy.table import Table
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import time
from sklearn import neighbors 
from vast.voidfinder.voidfinder_functions import not_in_mask
import pickle
from scipy.spatial import cKDTree
from vast.voidfinder import ra_dec_to_xyz

In [2]:
file_namevf_holes = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/dsim_200_128_t4_voids.txt"

In [3]:
data_table_vf_holes = Table.read(file_namevf_holes, format = "ascii.commented_header")

In [4]:
file_namevf_max = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/dsim_200_128_t4_maximals.txt"

In [5]:
data_table_vf_max = Table.read(file_namevf_max, format = "ascii.commented_header")

In [6]:
file_namegalaxies = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/out_200_128.dat"

In [7]:
data_table_galaxies = Table.read(file_namegalaxies, format = "ascii.commented_header")

In [8]:
file_namecn = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/CN_200_128.npy"

In [9]:
data_table_cn = np.load(file_namecn)

In [10]:
file_namegv = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/DR7_gv2.npy"

In [11]:
data_table_gv = np.load(file_namegv)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/DR7_gv2.npy'

In [ ]:
file_namevollim = "/Users/jordanconey/Documents/Summer_2021_REU/CN_vf_Comp/vollim_dr7_cbp_102709.dat"

In [ ]:
data_table_vl = Table.read(file_namevollim, format = "ascii.commented_header")

In [ ]:
data_table_vf_holes

In [ ]:
data_table_vf_max

In [ ]:
data_table_galaxies

## Add CN to `data_table_galaxies` column

In [ ]:
data_table_cn.shape

In [ ]:
data_table_galaxies['CN'] = data_table_cn
data_table_galaxies["X"].name = "x"
data_table_galaxies["Y"].name = "y"
data_table_galaxies["Z"].name = "z"

# Add GV to `data_table_vl` column

In [ ]:
data_table_vl['GV'] = data_table_gv


In [ ]:
galaxies_xyz = ra_dec_to_xyz(data_table_vl)


In [ ]:
data_table_vl['x'] = galaxies_xyz[:,0]
data_table_vl['y'] = galaxies_xyz[:,1]
data_table_vl['z'] = galaxies_xyz[:,2]
data_table_vl

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(data_table_vf_max['x'],data_table_vf_max['y'], data_table_vf_max['z'],
           color = 'green', s=10, alpha=0.5,
           label='max')
ax.set(xlabel='X [Mpc/h]',
       ylabel='Y [Mpc/h]',
       zlabel='Z [Mpc/h]')

ax.legend(loc='upper right', fontsize=10)
plt.title("Maximal Spheres of VoidFinder");

In [ ]:
# p_size = 100
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.scatter(data_table_vf_holes['x'],data_table_vf_holes['y'], data_table_vf_holes['z'],
           color = 'black', s=0.3, alpha=1.0,
           label='holes')
ax.scatter(data_table_vf_max['x'],data_table_vf_max['y'],data_table_vf_max['z'],
           color = 'gold', s=30, alpha=1.0,
           label='max')
ax.set(xlabel='X [Mpc/h]',
       ylabel='Y [Mpc/h]',
       zlabel='Z [Mpc/h]')

ax.legend(loc='upper right', fontsize=10)

plt.title("Maximal Sphere and Holes of VoidFinder");

In [ ]:
def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
    
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from first catalog.
    void_cat_B : astropy.Table
        Table of void data from second catalog.
        
    Returns
    -------
    x_min : float
    x_max : float
    y_min : float
    y_max : float
    z_min : float
    z_max : float
    """
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
    
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    return x_min, x_max, y_min, y_max, z_min, z_max

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
    
    Returns
    -------
    xyz : list
        2D list of points in 3D space.
    """
    x_range = np.arange(x_min, x_max)
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)

    # Creating a meshgrid from the ranges to 
    X,Y,Z = np.meshgrid(x_range,y_range,z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
    
    point_coords = np.array([x_points, y_points, z_points])
    
    return point_coords

In [ ]:
#This line creates the boundaries (like the whole min and max)
xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(data_table_vf_holes, data_table_vl)

#This line makes creates the points in between 
pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

b = pts.shape
print(b)

In [ ]:
# # This is the cell to filter out all the points that we do not want 🥴
# start_time = time.time()

# rmin = 0
# rmax = 312.89816

# points_boolean = np.ones(pts.shape[1], dtype = bool)

# print(time.time() - start_time)

# start_time = time.time()
# for i in range(pts.shape[1]):
    
#     #   print(pts[:,i].reshape((1,3)).shape)
#     #   print(pts[:,i].reshape((1,3)))
#     #   print(pts[:,i])
#     points_boolean[i] = not_in_mask(pts[:,i].reshape((1,3)), mask, mask_resolution, rmin, rmax)        


# points_in_mask = pts[:,~points_boolean]
# print(time.time() - start_time)
# print(points_in_mask.shape)
# print(np.sum(points_boolean))
# print(np.sum(~points_boolean))
# print(points_boolean.shape)

In [ ]:
# should give me my good cube shape
start_time = time.time()
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(pts[0,:],pts[1,:], pts[2,:],
           color = 'blue', s=10, alpha=0.5,
           label='max')
# ax.set(xlabel='X [Mpc/h]',
#        ylabel='Y [Mpc/h]',
#        zlabel='Z [Mpc/h]')

# plt.title("Points");
print(time.time() - start_time)

In [ ]:
def point_query(point_coords, void_cat,V2):
    """We are creating a function to make a KDTree to find the number of points in 
    and out of a catalogue.
    
    Parameters
    ----------
    point_coords: ndarray has a shape of (3,N)
        This is the list of points to query the given void catalogue. N is the number of points given. 
    void_cat: Astropy Table
        This is the given void catalogue.
    V2: boolean 
        This tells me if my catalog is a V2 catalog or not a V2 catalog.
    
    Returns
    -------
    true_inside: ndarray of shape (N,1)
        Is this the boolean array of length N (same length as point_coords). True means that 1 point 
        is inside the hole.
    """
    
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    sphere_coords = np.array([cx, cy, cz])

    start_time = time.time()

    #The .T is meant to transpose the array from (3,1054) to (1054,3)
    sphere_tree = neighbors.KDTree(sphere_coords.T)

    print(time.time() - start_time)

    start_time = time.time()

    dist, idx = sphere_tree.query(point_coords.T, k = 1)
    
    if not V2:

        true_inside = dist < void_cat['radius'][idx]
    
    else: 
        """What goes into the square braket is whatever the name of the column that tells
        me what is in a wall and what in a void
        
        May need adjust 
        """

        true_inside = void_cat["GV"][idx] == True
    
    return true_inside

In [ ]:
start_time = time.time()

true_inside_vf = point_query(pts, data_table_vf_holes,False)

count_in_vf = np.sum(true_inside_vf)

# The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
count_out_vf = np.sum(~true_inside_vf)
print(time.time() - start_time)

total_vf = count_in_vf + count_out_vf

true_inside_vl = point_query(pts, data_table_vl,True)

count_in_vl = np.sum(true_inside_vl)

# The "~" inverts the array. So we have true_inside inverted to add up the falses instead of the trues
count_out_vl = np.sum(~true_inside_vl)

total_vl = count_in_vl + count_out_vl

print(time.time() - start_time)
print('\nNumber of points inside VoidFinder: {}'.format(count_in_vf))
print('\nNumber of points outside VoidFinder: {}'.format(count_out_vf))
print('\nNumber of points inside V2: {}'.format(count_in_vl))
print('\nNumber of points outside V2: {}'.format(count_out_vl))
print("\nThis is the total number of points VoidFinder: {}".format(total_vf))
print("\nThis is the total number of points in V2: {}".format(total_vl))

## This is the number of points that are inside both A and B

In [ ]:
inside_A_and_B = np.logical_and(true_inside_vf, true_inside_CN)

np.sum(inside_A_and_B), count_in_vf, count_in_CN

## This is the number of points that are in neither A and B

In [ ]:
not_inside_A_and_B = np.logical_and(~true_inside_vf, ~true_inside_CN)

np.sum(not_inside_A_and_B), count_in_vf, count_in_CN

## This is the number of points that are in A but not B

In [ ]:
inside_A = np.logical_and(true_inside_vf, ~true_inside_CN)

np.sum(inside_A)

## This is the number of points that are not in A but are in B

In [ ]:
inside_B = np.logical_and(~true_inside_vf, true_inside_CN)

np.sum(inside_B)

# Fraction of points: Calculate the ratios of the given sums

- in A
- in B
- in A & B
- in neither A nor B
- {in A, not B
- {in B, not A
-   |
-   |
- \ /
- The closer they are to zero, the more similar they are

## This is the ratio of A

In [ ]:
r_A = count_in_vf / total_vf
print(r_A)

## This is the ratio of B

In [ ]:
r_B = count_in_CN / total_vf
print(r_B)

## This is the ratio of both A and B

In [ ]:
r_AB = np.sum(inside_A_and_B) / total_vf
print(r_AB)

## This is the ratio of neither A nor B

In [ ]:
r_notAB = np.sum(not_inside_A_and_B) / total_vf
print(r_notAB)

## This is the ratio of A and not B

In [ ]:
r_A_not_B = np.sum(inside_A) / total_vf
print(r_A_not_B)

## This is the ratio of B and not A

In [ ]:
r_B_not_A = np.sum(inside_B) / total_vf
print(r_B_not_A)

In [ ]:
Sum = r_AB + r_notAB + r_A_not_B + r_B_not_A

In [ ]:
Sum

# This will be the Standard Deviation and Mean of the Data


## This is the STD and Mean of KIAS


## This is the STD and Mean of KIAS
